<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm

train_mnist = datasets.MNIST("", train=True, transform=transforms.ToTensor(), download=True)

test_mnist = datasets.MNIST("", train=False, transform=transforms.ToTensor(), download=True)

train = []
test = []
for data in train_mnist:
  train.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])

for data in test_mnist:
  test.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])


print(len(train))
print(len(test))
print(len(train[0][0]), type(train[0][0]))

60000
10000
784 <class 'numpy.ndarray'>


In [51]:
np.random.seed(1)
num_input = 784
num_output = 10
weight = np.random.random((num_output, num_input))
bias = 0.0
learning_rate = 0.01

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def forward(x):
  net = [0] * num_output
  for i in range(num_output):
    for j in range(num_input):
      net[i] += (x[j] * weight[i][j] + bias)
  return softmax(net)

# MSE
def calc_loss(y, y_pred):
  #print(y)
  #print(y_pred)
  return ((y-y_pred)**2).mean()

# gradient of MSE dJ/dw = 1/N 2x (w*x - y)
def calc_gradient(x, y, y_pred):
  grad_list = []
  for i in range(len(y)):
    grad_list.append((np.dot(2*x, y_pred[i] - y[i])).mean())
  return grad_list

#print("prediction before training : ", forward(10))

for epoch in range(5):
  BATCH_SIZE = 20 # without replacement
  grad_list_batch = [0]*num_output
  num_iter=0
  for data in tqdm(train):
    num_iter+=1
    x, y = data[0], data[1]
    y_pred = forward(x, y)
    #print(sum(y_pred))
    loss = calc_loss(y, y_pred)

    grad = calc_gradient(x, y, y_pred)

    grad_list_batch = [sum(x) for x in zip(grad_list_batch, grad)]

    if(num_iter%BATCH_SIZE==0):
      grad_list_batch = [x/BATCH_SIZE for x in grad_list_batch]
      for i in range(num_output):
        for j in range(num_input):
          weight[i][j]-=(learning_rate*grad_list_batch[i])
      grad_list_batch = [0]*num_output

  print("epoch : ", epoch, "loss : ", loss)

  0%|          | 4/60000 [00:00<25:03, 39.90it/s]

epoch :  0 loss :  0.1747928258134186


  0%|          | 4/60000 [00:00<26:08, 38.24it/s]

epoch :  1 loss :  0.17479282581341857


  0%|          | 4/60000 [00:00<25:23, 39.37it/s]

epoch :  2 loss :  0.17479282581341848


  0%|          | 4/60000 [00:00<25:38, 39.01it/s]

epoch :  3 loss :  0.17479282581341832


 91%|█████████ | 54453/60000 [24:13<02:28, 37.26it/s]

Buffered data was truncated after reaching the output size limit.

In [58]:
correct = 0
for data in tqdm(test):
    x, y = data[0], data[1]
    y_pred = forward(x, y)
    if(np.argmax(y_pred) == np.argmax(y)):
      correct+=1

print(correct/len(test))

100%|██████████| 10000/10000 [04:20<00:00, 38.41it/s]

0.1247
